In [41]:
from nlp_common.acts_reader import ActsReader
import regex
import requests
import pandas as pd
from collections import Counter
import numpy as np
import os.path

In [42]:
reader = ActsReader('../ustawy')
bills = [ act[2] for act in reader.all_acts() ]

In [43]:
new_line_re = regex.compile(r'\n+|\s+|\t+')
bills = [new_line_re.sub(' ', bill) for bill in bills]

### Tagging

In [44]:
def get_tags():
    file_name = "tags_response.txt"
    if os.path.exists(file_name):
        print("Reading cached response")
        with open(file_name, "r") as f:
            return f.read()
    else:
        with open(file_name, "w") as f:
            for i, bill in enumerate(bills):
                tagged_response = requests.post('http://localhost:9200', bill.encode(encoding='utf-8'))
                f.write(tagged_response.text)
                print(f"Processed: {i+1}/{len(bills)}")
        return get_tags()

tagged_response = get_tags()

Processed: 1/1179
Processed: 2/1179
Processed: 3/1179
Processed: 4/1179
Processed: 5/1179
Processed: 6/1179
Processed: 7/1179
Processed: 8/1179
Processed: 9/1179
Processed: 10/1179
Processed: 11/1179
Processed: 12/1179
Processed: 13/1179
Processed: 14/1179
Processed: 15/1179
Processed: 16/1179
Processed: 17/1179
Processed: 18/1179
Processed: 19/1179
Processed: 20/1179
Processed: 21/1179
Processed: 22/1179
Processed: 23/1179
Processed: 24/1179
Processed: 25/1179
Processed: 26/1179
Processed: 27/1179
Processed: 28/1179
Processed: 29/1179
Processed: 30/1179
Processed: 31/1179
Processed: 32/1179
Processed: 33/1179
Processed: 34/1179
Processed: 35/1179
Processed: 36/1179
Processed: 37/1179
Processed: 38/1179
Processed: 39/1179
Processed: 40/1179
Processed: 41/1179
Processed: 42/1179
Processed: 43/1179
Processed: 44/1179
Processed: 45/1179
Processed: 46/1179
Processed: 47/1179
Processed: 48/1179
Processed: 49/1179
Processed: 50/1179
Processed: 51/1179
Processed: 52/1179
Processed: 53/1179
Pr

Processed: 417/1179
Processed: 418/1179
Processed: 419/1179
Processed: 420/1179
Processed: 421/1179
Processed: 422/1179
Processed: 423/1179
Processed: 424/1179
Processed: 425/1179
Processed: 426/1179
Processed: 427/1179
Processed: 428/1179
Processed: 429/1179
Processed: 430/1179
Processed: 431/1179
Processed: 432/1179
Processed: 433/1179
Processed: 434/1179
Processed: 435/1179
Processed: 436/1179
Processed: 437/1179
Processed: 438/1179
Processed: 439/1179
Processed: 440/1179
Processed: 441/1179
Processed: 442/1179
Processed: 443/1179
Processed: 444/1179
Processed: 445/1179
Processed: 446/1179
Processed: 447/1179
Processed: 448/1179
Processed: 449/1179
Processed: 450/1179
Processed: 451/1179
Processed: 452/1179
Processed: 453/1179
Processed: 454/1179
Processed: 455/1179
Processed: 456/1179
Processed: 457/1179
Processed: 458/1179
Processed: 459/1179
Processed: 460/1179
Processed: 461/1179
Processed: 462/1179
Processed: 463/1179
Processed: 464/1179
Processed: 465/1179
Processed: 466/1179


Processed: 827/1179
Processed: 828/1179
Processed: 829/1179
Processed: 830/1179
Processed: 831/1179
Processed: 832/1179
Processed: 833/1179
Processed: 834/1179
Processed: 835/1179
Processed: 836/1179
Processed: 837/1179
Processed: 838/1179
Processed: 839/1179
Processed: 840/1179
Processed: 841/1179
Processed: 842/1179
Processed: 843/1179
Processed: 844/1179
Processed: 845/1179
Processed: 846/1179
Processed: 847/1179
Processed: 848/1179
Processed: 849/1179
Processed: 850/1179
Processed: 851/1179
Processed: 852/1179
Processed: 853/1179
Processed: 854/1179
Processed: 855/1179
Processed: 856/1179
Processed: 857/1179
Processed: 858/1179
Processed: 859/1179
Processed: 860/1179
Processed: 861/1179
Processed: 862/1179
Processed: 863/1179
Processed: 864/1179
Processed: 865/1179
Processed: 866/1179
Processed: 867/1179
Processed: 868/1179
Processed: 869/1179
Processed: 870/1179
Processed: 871/1179
Processed: 872/1179
Processed: 873/1179
Processed: 874/1179
Processed: 875/1179
Processed: 876/1179


### Bigrams

In [46]:
def create_unigrams(tagged_corpus):
    splitted = [ l for l in tagged_corpus.split('\n') if l != '']
    unigrams = []
    for i in range(0, len(splitted), 2):
        if splitted[i].startswith('\t') or not splitted[i+1].startswith('\t'):
            raise Exception("Wrong assumption")
            
        tagging =  splitted[i+1].split('\t')
        tags = tagging[2].split(':')
        unigrams.append(f'{tagging[1].lower()}:{tags[0]}')
    
    return unigrams
    
unigrams = create_unigrams(tagged_response)

In [47]:
def create_bigrams(unigrams):
    return [(unigrams[i], unigrams[i+1]) for i in range(len(unigrams)-1)]
    
bigrams = create_bigrams(unigrams)

### LLR

In [48]:
def compute_global_counter(ctrs): 
    global_counter = Counter()
    for ctr in ctrs:
        global_counter += ctr
    return global_counter

def probabilities(ngrams, filter_fn=None):
    ctrs = [Counter(doc) for doc in ngrams]
    global_counter = compute_global_counter(ctrs)
    l = len(global_counter)
    probs = { k: global_counter[k]/l for k in global_counter 
            if filter_fn == None or filter_fn(k, global_counter[k])}
    global_counter = { k: global_counter[k] for k in global_counter
            if filter_fn == None or filter_fn(k, global_counter[k]) }
    return probs, global_counter

def probabilities_flat(counter_ngrams, map_key_fn):
    ctr = Counter()
    for k in counter_ngrams:
        mk = map_key_fn(k)
        ctr[mk] += counter_ngrams[k]
    l = len(ctr)
    probs = {
        k: ctr[k]/l for k in ctr
    }
    return probs, ctr

def pmi_df_bigram(bill_docs_bigrams, filter_fn=None):
    pm, pmctr = probabilities(bill_docs_bigrams, filter_fn)
    pa, pactr = probabilities_flat(pmctr, lambda x: x[0])
    pb, pbctr = probabilities_flat(pmctr, lambda x: x[1])

    df = pd.DataFrame(data=[(k,) for k in pm.keys()], columns = ["bigram"])
    df["p(a)"] = df["bigram"].apply(lambda x: pa[x[0]])
    df["#a"] = df["bigram"].apply(lambda x: pactr[x[0]])
    df["p(b)"] = df["bigram"].apply(lambda x: pb[x[1]])
    df["#b"] = df["bigram"].apply(lambda x: pbctr[x[1]])
    df["p(a,b)"] = df["bigram"].apply(lambda x: pm[x])
    df["#ab"] = df["bigram"].apply(lambda x: pmctr[x])
    df["pmi"] = np.log(df["p(a,b)"]/ (df["p(a)"]*df["p(b)"]))
    return df#.sort_values(["pmi"], ascending=[0])

def H(a):
    a = np.array(a)
    N = a.sum()
    return np.sum(a/N * np.log(a/N + (a==0)))
                                  
def llr(pmi_df):
    def apply_llr(k):
        return 2*np.sum(k)*(H(k) - H(k.sum(axis=0)) - H(k.sum(axis=1)))
        
    df = pmi_df.copy()
    sum_all = pmi_df["#ab"].sum()
    df["k11"] = pmi_df["#ab"]
    df["k12"] = pmi_df["#a"] - df["k11"]
    df["k21"] = pmi_df["#b"] - df["k11"] 
    df["k22"] = sum_all - (pmi_df["#a"] + pmi_df["#b"] - pmi_df["#ab"])
    df["k"] = df[["k11", "k12","k21", "k22"]].values.tolist()
    df["k"] = df["k"].apply(lambda k: np.array(k).reshape((2,2)))
    df["llr"] = df["k"].apply(apply_llr)
    pmi_df["llr"] = df["llr"]
    return pmi_df.sort_values(["llr"], ascending=[0])

In [49]:
bigram_regex = regex.compile('\p{L}+:\p{L}+')
llrdf = llr(pmi_df_bigram([bigrams], lambda k,_: bigram_regex.match(k[0]) and bigram_regex.match(k[1])))

In [50]:
llrdf.head(30)

,bigram,p(a),#a,p(b),#b,"p(a,b)",#ab,pmi,llr
605,"(który:adj, mowa:subst)",2.331399,49006,1.315189,28800,0.060105,28538,-3.932108,248056.086445
604,"(o:prep, który:adj)",3.051951,64152,1.701251,37254,0.060354,28656,-4.454678,190484.834600
606,"(mowa:subst, w:prep)",1.368839,28773,5.936022,129987,0.059968,28473,-4.908942,177065.653893
178,"(w:prep, artykuł:brev)",9.442150,198474,2.189515,47946,0.067390,31997,-5.726117,114479.407925
27,"(otrzymywać:fin, brzmienie:subst)",0.520124,10933,0.815143,17850,0.022188,10535,-2.950112,110746.706564
611,"(w:prep, ustęp:brev)",9.442150,198474,1.512284,33116,0.049547,23525,-5.663637,87902.958510
709,"(minister:subst, właściwy:adj)",0.697431,14660,0.600009,13139,0.016708,7933,-3.220702,70825.836306
180,"(dodawać:fin, się:qub)",0.400571,8420,2.094347,45862,0.017262,8196,-3.883627,66616.945839
6487,"(i:conj, numer:brev)",3.676213,77274,0.432779,9477,0.017765,8435,-4.494862,54715.014682
962,"(droga:subst, rozporządzenie:subst)",0.379591,7979,0.261394,5724,0.009998,4747,-2.294994,53905.637186


## Partition

In [51]:
def syntactic_category(bigram):
    return (bigram[0].split(':')[1],bigram[1].split(':')[1])

def syntactic_partition(llrdf):
    partitions = {}
    for bigram, llr in llrdf[["bigram", "llr"]].values:
        cat = syntactic_category(bigram)
        if not cat in partitions:
            partitions[cat] = [(bigram, llr)]
        else:
            partitions[cat].append((bigram, llr))
    return partitions    

In [52]:
partitions = syntactic_partition(llrdf)

In [53]:
pdf = pd.DataFrame(data=[(k, len(partitions[k])) for k in partitions.keys()], columns = ["category", "#"])
pdf = pdf.sort_values(["#"], ascending=[0])
pdf = pdf[:10]
pdf["best 5 llr"] = pdf["category"].apply(lambda x: sorted(partitions[x], key=lambda d: -d[1])[:5] )

In [54]:
from pandas import option_context

with option_context('display.max_colwidth', 400):
    display(pdf.head())

,category,#,best 5 llr
8,"(subst, subst)",45998,"[((droga:subst, rozporządzenie:subst), 53905.63718592001), ((skarb:subst, państwo:subst), 22069.49101682054), ((rada:subst, minister:subst), 18306.81920969008), ((terytorium:subst, rzeczpospolita:subst), 14073.049996732097), ((ochrona:subst, środowisko:subst), 14016.747437113545)]"
5,"(subst, adj)",26877,"[((minister:subst, właściwy:adj), 70825.83630634959), ((rzeczpospolita:subst, polski:adj), 42184.92278997753), ((jednostka:subst, organizacyjny:adj), 24500.405325953747), ((samorząd:subst, terytorialny:adj), 23434.858141620585), ((produkt:subst, leczniczy:adj), 21899.349704130764)]"
0,"(adj, subst)",26042,"[((który:adj, mowa:subst), 248056.086444919), ((niniejszy:adj, ustawa:subst), 21432.655987571314), ((następujący:adj, zmiana:subst), 20452.04406757462), ((odrębny:adj, przepis:subst), 12944.905989283785), ((walny:adj, zgromadzenie:subst), 9466.245471411135)]"
29,"(subst, fin)",16176,"[((kropka:subst, zastępować:fin), 10232.51339858724), ((ustawa:subst, wchodzić:fin), 9366.554104666468), ((treść:subst, oznaczać:fin), 2851.857280933342), ((minister:subst, określić:fin), 2702.18962263325), ((zdrowie:subst, określić:fin), 1489.3163315592155)]"
24,"(ger, subst)",15880,"[((pozbawić:ger, wolność:subst), 14634.017533219916), ((zasięgnąć:ger, opinia:subst), 11520.543605066367), ((wykonywać:ger, zawód:subst), 5551.118093470538), ((zawrzeć:ger, umowa:subst), 5190.121733924526), ((wszcząć:ger, postępowanie:subst), 5141.536035725716)]"


### Questions

##### What types of bigrams have been found?
Mostly pairs of type (noun,noun) and many of them are case governments (związek rządu). The phrases appear more often right because of lematization and all different forms are counted together.

##### Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?
The ones with bigger counts?

##### Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?
It definitely depends on the problem that we solve and what we want to find. They will we applicable for different problems. They both works well together as the last task have shown.

##### Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?
By knowing the rules what follows the words usually, we are able to for examoke build sentences that are answers for questions.